In [913]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ML_utils import load_data,load_data_test_set, total_day_eve_night_grouping
df =pd.read_csv('train.csv')

### UNCOMMENT THIS PART TO GROUP FEATURES
feature_engineering = True
df=total_day_eve_night_grouping(df,grouping=feature_engineering)

exclude_list=['state','area_code']

# exclude_list = [feature_name for feature_name in df.columns if feature_name not in['number_vmail_messages','number_customer_service_calls','total_charges','churn']]
one_hot_on = True
normalize_on = True
oversample_on=False


###################
#### LOAD DATA ####

X_train,y_train,X_val,y_val,X_test,y_test,columns = load_data(df,exclude=exclude_list,one_hot=one_hot_on,normalize_=normalize_on,oversample=oversample_on)

X_train.shape

# X_train = np.concatenate((X_train, X_train[:,0].reshape(-1,1)), axis=1)
# X_train[:,-1]=X_train[:,-1]**2
# columns.insert(-1,'total_charges^2')

(2975, 11)

In [914]:
from annoy import AnnoyIndex



index =AnnoyIndex(X_train.shape[1], 'euclidean') 

number_of_trees = [10,20,50]
k = [5,10,20]
slack_list = [0.5,0.7,2]



# Length of item vector that will be indexed
for i, vector in enumerate(X_train):
    index.add_item(i, vector)


number_of_trees = 10
k = 5

index.build(number_of_trees) # 10 trees




True

In [919]:

def get_knn(index, X, k):
    knn = []
    for vector in X:
        knn.append(index.get_nns_by_vector(vector, k))
    return knn





def predict(X):
    knn = []
    for vector in X:
        knn.append(index.get_nns_by_vector(vector, k))
    y_pred=[]
    for i in knn:
        y_pred.append(np.mean(y_train[i]))
        
    return np.round(y_pred)


y_pred=predict(X_val)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [916]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(classification_report(y_val,y_pred))

print(accuracy_score(y_val,y_pred))


              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       562
         1.0       0.89      0.45      0.60        75

    accuracy                           0.93       637
   macro avg       0.91      0.72      0.78       637
weighted avg       0.93      0.93      0.92       637

0.9293563579277865


In [917]:
### USE THE MODEL ON THE TEST SET
df_test=pd.read_csv('test.csv')

df_test=total_day_eve_night_grouping(df_test,grouping=feature_engineering)


X_test_output=load_data_test_set(df_test,exclude=exclude_list+['id'],one_hot=one_hot_on,normalize_=normalize_on)

y_pred_output=predict(X_test_output)

y_pred_output=np.where(y_pred_output>0.5,'yes', 'no')
y_pred_output=y_pred_output.reshape(-1,)
id_column = np.arange(1, y_pred_output.shape[0] + 1)


# Create a DataFrame
df_output = pd.DataFrame({
    'id': id_column,
    'churn': y_pred_output
})


# Save the DataFrame as a CSV file
df_output.to_csv('output_knn_annoy.csv', index=False)